In [1]:
import pandas as pd
import numpy as np
import os
import re
import copy
import ast
import warnings
warnings.filterwarnings("ignore")
from icecream import ic

random_state = 0

## Load the data

In [8]:
random_state = 0
nacc = pd.read_csv('training_cohorts/new_nacc_revised_selection.csv')
nacc = nacc.sample(frac=1, random_state=random_state).reset_index(drop=True)

nacc_np = pd.read_csv('train_vld_test_split_updated/nacc_neuropath.csv')
nacc_np = nacc[nacc['ID'].isin(nacc_np['ID'])]

cli = pd.read_csv('train_vld_test_split_updated/clinician_review_cases_test.csv')
cli = nacc[nacc['ID'].isin(cli['ID'])]

nacc_wo_np_cli = nacc[(~nacc['ID'].isin(nacc_np['ID'])) & (~nacc['ID'].isin(cli['ID']))]

merged = pd.read_csv('training_cohorts/merged_data_nacc_nifd_stanford_aibl_ppmi_oasis_rtni_without_np_cli.csv')
merged = merged.sample(frac=1, random_state=random_state).reset_index(drop=True)

test = pd.read_csv('train_vld_test_split_updated/nacc_test_with_np_cli.csv')

In [9]:
len(merged[merged['ID'].str.startswith('AIBL') & merged['cdr_CDRGLOB'].isna()])

0

In [10]:
test

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip
0,NACC900273,258,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,4.5,NaN
1,NACC598453,193,4.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.0,mri6866.zip
2,NACC448699,193,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.0,NaN
3,NACC996281,185,5.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,16.0,mri5135.zip
4,NACC797956,193,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8890,NACC856668,263,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.5,3.5,NaN
8891,NACC941694,192,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.5,3.5,NaN
8892,NACC093272,209,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,7.0,NaN
8893,NACC174114,208,NaN,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.5,NaN


## If test file is present

### NACC

In [16]:
nacc_trn_vld = nacc_wo_np_cli[~nacc_wo_np_cli['ID'].isin(test['ID'])]

In [17]:
def get_train_split(df, save_path):
    nc = df[df['NC'] == 1][:int(0.75 * len(df[df['NC'] == 1]))]
    # imci = df[df['IMCI'] == 1][:int(0.6 * len(df[df['IMCI'] == 1]))]
    mci = df[df['MCI'] == 1][:int(0.75 * len(df[df['MCI'] == 1]))]
    # de = nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1][:int(0.8 * len(nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1]))]
    ad = df[df['AD'] == 1][:int(0.75 * len(df[df['AD'] == 1]))]
    lbd = df[df['LBD'] == 1][:int(0.75 * len(df[df['LBD'] == 1]))]
    vd = df[df['VD'] == 1][:int(0.68 * len(df[df['VD'] == 1]))]
    prion = df[df['PRD'] == 1][:int(0.7 * len(df[df['PRD'] == 1]))]
    ftd = df[df['FTD'] == 1][:int(0.75 * len(df[df['FTD'] == 1]))]
    nph = df[df['NPH'] == 1][:int(0.75 * len(df[df['NPH'] == 1]))]
    sef = df[df['SEF'] == 1][:int(0.75 * len(df[df['SEF'] == 1]))]
    psy = df[df['PSY'] == 1][:int(0.75 * len(df[df['PSY'] == 1]))]
    tbi = df[df['TBI'] == 1][:int(0.55 * len(df[df['TBI'] == 1]))]
    oc = df[df['ODE'] == 1][:int(0.75 * len(df[df['ODE'] == 1]))]

    df_train = pd.concat([nc, mci, ad, lbd, vd, prion, ftd, nph, sef, psy, tbi, oc], ignore_index=True)
    df_train = df_train.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_train.drop_duplicates(inplace=True)
    df_train.to_csv(save_path, index=False)
    return df_train

def get_vld_split(df, save_path, df_train):
    df_vld = df[~df['ID'].isin(list(df_train['ID']))]
    df_vld = df_vld.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_vld.drop_duplicates(inplace=True)
    df_vld.to_csv(save_path, index=False)
    return df_vld
    

In [18]:
train = get_train_split(nacc_trn_vld, 'train_vld_test_split_updated/nacc_train.csv')
vld = get_vld_split(nacc_trn_vld, 'train_vld_test_split_updated/nacc_vld.csv', train)
# # test.to_csv('training_cohorts/train_vld_test_split/nacc_test.csv', index=False)

In [19]:
len(set(train['ID'])) + len(set(vld['ID'])) + len(set(test['ID']))

45349

In [20]:
len(nacc)

45349

In [21]:
set(vld['ID']).intersection(set(train['ID']))

set()

In [22]:
set(train['mri_zip']).intersection(set(nacc_np['mri_zip']).union(set(cli['mri_zip'])))

set()

In [23]:
nacc_lab_dict = {}
nacc_trn_lab_dict = {}
nacc_vld_lab_dict = {}
nacc_tst_lab_dict = {}
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    nacc_lab_dict[label] = nacc[label].value_counts()
    nacc_trn_lab_dict[label] = train[label].value_counts()
    nacc_vld_lab_dict[label] = vld[label].value_counts()
    nacc_tst_lab_dict[label] = test[label].value_counts()
train_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_trn_lab_dict.items()}
vld_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_vld_lab_dict.items()}
test_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_tst_lab_dict.items()}

In [24]:
train_ratios

{'NC': 0.6053822062405754,
 'MCI': 0.6030071221313638,
 'DE': 0.6021437271619976,
 'AD': 0.602876449073213,
 'LBD': 0.5938316779926817,
 'VD': 0.6060448150078166,
 'PRD': 0.5877192982456141,
 'FTD': 0.6080055210489993,
 'NPH': 0.6159420289855072,
 'SEF': 0.6002475247524752,
 'PSY': 0.607037037037037,
 'TBI': 0.6037735849056604,
 'ODE': 0.6045380875202593}

In [25]:
vld_ratios

{'NC': 0.20183273402157523,
 'MCI': 0.20100237404378793,
 'DE': 0.19883069427527406,
 'AD': 0.1993056847064658,
 'LBD': 0.1970726607422896,
 'VD': 0.20114643043251693,
 'PRD': 0.2631578947368421,
 'FTD': 0.19703243616287094,
 'NPH': 0.18840579710144928,
 'SEF': 0.2004950495049505,
 'PSY': 0.20074074074074075,
 'TBI': 0.19245283018867926,
 'ODE': 0.20178282009724474}

In [26]:
test_ratios

{'NC': 0.19278505973784943,
 'MCI': 0.1959905038248483,
 'DE': 0.19902557856272837,
 'AD': 0.19781786622032113,
 'LBD': 0.20909566126502874,
 'VD': 0.1928087545596665,
 'PRD': 0.14912280701754385,
 'FTD': 0.19496204278812976,
 'NPH': 0.1956521739130435,
 'SEF': 0.19925742574257427,
 'PSY': 0.1922222222222222,
 'TBI': 0.2037735849056604,
 'ODE': 0.19367909238249595}

In [27]:
merged[merged['ID'].isna()]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


### Merged data split

In [28]:
merged_train_vld = merged[~merged['ID'].isin(list(test['ID']))]

In [29]:
def get_train_split_merged(df, save_path):
    nc = df[df['NC'] == 1][:int(0.75 * len(df[df['NC'] == 1]))]
    mci = df[df['MCI'] == 1][:int(0.75 * len(df[df['MCI'] == 1]))]
    ad = df[df['AD'] == 1][:int(0.75 * len(df[df['AD'] == 1]))]
    lbd = df[df['LBD'] == 1][:int(0.75 * len(df[df['LBD'] == 1]))]
    vd = df[df['VD'] == 1][:int(0.68 * len(df[df['VD'] == 1]))]
    prion = df[df['PRD'] == 1][:int(0.7 * len(df[df['PRD'] == 1]))]
    ftd = df[df['FTD'] == 1][:int(0.75 * len(df[df['FTD'] == 1]))]
    nph = df[df['NPH'] == 1][:int(0.75 * len(df[df['NPH'] == 1]))]
    sef = df[df['SEF'] == 1][:int(0.75 * len(df[df['SEF'] == 1]))]
    psy = df[df['PSY'] == 1][:int(0.75 * len(df[df['PSY'] == 1]))]
    tbi = df[df['TBI'] == 1][:int(0.75 * len(df[df['TBI'] == 1]))]
    oc = df[df['ODE'] == 1][:int(0.75 * len(df[df['ODE'] == 1]))]

    df_train = pd.concat([nc, mci, ad, lbd, vd, prion, ftd, nph, sef, psy, tbi, oc], ignore_index=True)
    df_train = df_train.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_train.drop_duplicates(inplace=True)
    df_train.to_csv(save_path, index=False)
    return df_train

def get_vld_split_merged(df, save_path, df_train):
    df_vld = df[~df['ID'].isin(list(df_train['ID']))]
    df_vld = df_vld.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_vld.drop_duplicates(inplace=True)
    df_vld.to_csv(save_path, index=False)
    return df_vld

In [30]:
merged_train = get_train_split_merged(merged_train_vld, 'train_vld_test_split_updated/merged_train.csv')
merged_vld = get_vld_split_merged(merged_train_vld, 'train_vld_test_split_updated/merged_vld.csv', merged_train)

#### tests

In [31]:
merged_train[(merged_train['NC'] == 0) & (merged_train['MCI'] == 0) & (merged_train['DE'] == 0)]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [32]:
merged_vld[(merged_vld['NC'] == 0) & (merged_vld['MCI'] == 0) & (merged_vld['DE'] == 0)]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [33]:
test[(test['NC'] == 0) & (test['MCI'] == 0) & (test['DE'] == 0)]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [34]:
merged_train[merged_train['DE'].isna()]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [35]:
merged_vld[merged_vld['DE'].isna()]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [36]:
test[test['DE'].isna()]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip


In [37]:
len(merged_train)

28760

In [38]:
len(merged_vld)

9559

In [39]:
len((merged_train['ID'])) + len((merged_vld['ID'])) + len((test['ID']))

47214

In [40]:
len(set(merged['ID'])) + len(set(nacc_np['ID']).union(set(cli['ID'])))

47214

In [41]:
set(merged_vld['ID']).intersection(set(test['ID']))

set()

In [42]:
set(merged['mri_zip']).intersection(set(nacc_np['mri_zip']).union(set(cli['mri_zip'])))

set()

In [43]:
merged_lab_dict = {}
merged_trn_lab_dict = {}
merged_vld_lab_dict = {}
merged_tst_lab_dict = {}
merged_ = pd.concat([merged, cli, nacc_np], ignore_index=True).sample(frac=1, random_state=random_state).reset_index(drop=True)
merged_.drop_duplicates(inplace=True)
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    merged_lab_dict[label] = merged_[label].value_counts()
    merged_trn_lab_dict[label] = merged_train[label].value_counts()
    merged_vld_lab_dict[label] = merged_vld[label].value_counts()
    merged_tst_lab_dict[label] = test[label].value_counts()
merged_train_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_trn_lab_dict.items()}
merged_vld_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_vld_lab_dict.items()}
merged_test_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_tst_lab_dict.items()}

In [44]:
merged_train_ratios

{'NC': 0.6158605476946253,
 'MCI': 0.6071245515120451,
 'DE': 0.6038995341689478,
 'AD': 0.6041127939908878,
 'LBD': 0.5953367875647668,
 'VD': 0.5977071391349661,
 'PRD': 0.5964912280701754,
 'FTD': 0.6117992177314211,
 'NPH': 0.6014492753623188,
 'SEF': 0.6027227722772277,
 'PSY': 0.6074074074074074,
 'TBI': 0.5962264150943396,
 'ODE': 0.6045380875202593}

In [45]:
merged_vld_ratios

{'NC': 0.2053047829127885,
 'MCI': 0.20246027678113787,
 'DE': 0.1999231618882966,
 'AD': 0.1994212535402044,
 'LBD': 0.19740932642487047,
 'VD': 0.20948410630536737,
 'PRD': 0.2543859649122807,
 'FTD': 0.20404172099087353,
 'NPH': 0.2028985507246377,
 'SEF': 0.19801980198019803,
 'PSY': 0.20037037037037037,
 'TBI': 0.2,
 'ODE': 0.20178282009724474}

In [46]:
merged_test_ratios

{'NC': 0.17883466939258622,
 'MCI': 0.19041517170681702,
 'DE': 0.1961773039427556,
 'AD': 0.19646595246890777,
 'LBD': 0.20725388601036268,
 'VD': 0.1928087545596665,
 'PRD': 0.14912280701754385,
 'FTD': 0.18415906127770534,
 'NPH': 0.1956521739130435,
 'SEF': 0.19925742574257427,
 'PSY': 0.1922222222222222,
 'TBI': 0.2037735849056604,
 'ODE': 0.19367909238249595}

In [47]:
merged_vld[(merged_vld['TBI'] == 1) & (~merged_vld['mri_zip'].isna())]

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip
1226,NACC786466,266.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.5,mri9492.zip
1829,NACC354005,199.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.5,3.5,mri8569.zip
4438,NACC566106,198.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,10.0,mri8536.zip
6115,NACC859532,194.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,6.0,NACC859532_12840113837269926852711609981130.zip
6843,NACC215530,200.0,5.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.0,NACC215530_12840113845111000000001814148435200...
6873,NACC904318,190.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,6.5,mri6766.zip
8616,NACC646210,211.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,3.0,mri9787.zip
9378,NACC113051,193.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.5,4.0,mri7889.zip


In [48]:
nacc_np

,ID,non_null_count,MRI_CNT,NC,MCI,DE,AD,LBD,PDD,DLB,...,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,cdr_CDRGLOB,cdr_CDRSUM,mri_zip
689,NACC191311,211,1.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,8.0,NACC191311_13122110752436602730000016062311441...
871,NACC073093,186,3.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,8.0,NACC073093_12840113619218210808614785218142879...
995,NACC747417,200,2.0,0,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,9.0,mri5510.zip
1070,NACC893973,261,4.0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NACC893973_13122110752323540730000016121313383...
1355,NACC092622,180,4.0,0,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,11.0,NACC092622_12840113619213417628947111858133130...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44240,NACC959661,194,1.0,0,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,13.0,NACC959661_128401143562013722104026376.zip
44528,NACC683293,132,2.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,18.0,NACC683293_13122110752436602730000017071111532...
44822,NACC077349,203,2.0,0,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.5,mri5558.zip
45140,NACC198306,199,4.0,0,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,4.5,mri392.zip


In [59]:
# tmp = pd.read_csv('/home/skowshik/ADRD_repo/data/training_cohorts/merged_data_nacc_nifd_stanford_aibl_ppmi_oasis_rtni_without_np_cli.csv')
len(tmp[~tmp['ID'].str.startswith('4RTNI')])

46764

In [60]:
len(merged[~merged['ID'].str.startswith('4RTNI')])

46764

## if test file not present

#### NACC

In [ ]:
def get_train_split(df, save_path):
    nc = df[df['NC'] == 1][:int(0.6 * len(df[df['NC'] == 1]))]
    # imci = df[df['IMCI'] == 1][:int(0.6 * len(df[df['IMCI'] == 1]))]
    mci = df[df['MCI'] == 1][:int(0.6 * len(df[df['MCI'] == 1]))]
    # de = nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1][:int(0.8 * len(nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1]))]
    ad = df[df['AD'] == 1][:int(0.6 * len(df[df['AD'] == 1]))]
    lbd = df[df['LBD'] == 1][:int(0.6 * len(df[df['LBD'] == 1]))]
    vd = df[df['VD'] == 1][:int(0.6 * len(df[df['VD'] == 1]))]
    prion = df[df['PRD'] == 1][:int(0.7 * len(df[df['PRD'] == 1]))]
    ftd = df[df['FTD'] == 1][:int(0.6 * len(df[df['FTD'] == 1]))]
    nph = df[df['NPH'] == 1][:int(0.6 * len(df[df['NPH'] == 1]))]
    sef = df[df['SEF'] == 1][:int(0.6 * len(df[df['SEF'] == 1]))]
    psy = df[df['PSY'] == 1][:int(0.6 * len(df[df['PSY'] == 1]))]
    tbi = df[df['TBI'] == 1][:int(0.5 * len(df[df['TBI'] == 1]))]
    oc = df[df['ODE'] == 1][:int(0.6 * len(df[df['ODE'] == 1]))]

    df_train = pd.concat([nc, mci, ad, lbd, vd, prion, ftd, nph, sef, psy, tbi, oc], ignore_index=True)
    df_train = df_train.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_train.drop_duplicates(inplace=True)
    # df_train.to_csv(save_path, index=False)
    return df_train

def get_vld_split(df, save_path, df_train):
    nc = df[df['NC'] == 1][int(0.6 * len(df[df['NC'] == 1])):int(0.81 * len(df[df['NC'] == 1]))]
    # imci = df[df['IMCI'] == 1][int(0.6 * len(df[df['IMCI'] == 1])):int(0.81 * len(df[df['IMCI'] == 1]))]
    mci = df[df['MCI'] == 1][int(0.6 * len(df[df['MCI'] == 1])):int(0.81 * len(df[df['MCI'] == 1]))]
    # de = nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1][:int(0.8 * len(nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1]))]
    ad = df[df['AD'] == 1][int(0.6 * len(df[df['AD'] == 1])):int(0.81 * len(df[df['AD'] == 1]))]
    lbd = df[df['LBD'] == 1][int(0.6 * len(df[df['LBD'] == 1])):int(0.81 * len(df[df['LBD'] == 1]))]
    vd = df[df['VD'] == 1][int(0.6 * len(df[df['VD'] == 1])):int(0.7 * len(df[df['VD'] == 1]))]
    prion = df[df['PRD'] == 1][int(0.7 * len(df[df['PRD'] == 1])):int(0.9 * len(df[df['PRD'] == 1]))]
    ftd = df[df['FTD'] == 1][int(0.6 * len(df[df['FTD'] == 1])):int(0.81 * len(df[df['FTD'] == 1]))]
    nph = df[df['NPH'] == 1][int(0.6 * len(df[df['NPH'] == 1])):int(0.85 * len(df[df['NPH'] == 1]))]
    sef = df[df['SEF'] == 1][int(0.6 * len(df[df['SEF'] == 1])):int(0.81 * len(df[df['SEF'] == 1]))]
    psy = df[df['PSY'] == 1][int(0.6 * len(df[df['PSY'] == 1])):int(0.81 * len(df[df['PSY'] == 1]))]
    tbi = df[df['TBI'] == 1][int(0.5 * len(df[df['TBI'] == 1])):int(0.6 * len(df[df['TBI'] == 1]))]
    oc = df[df['ODE'] == 1][int(0.6 * len(df[df['ODE'] == 1])):int(0.81 * len(df[df['ODE'] == 1]))]

    df_vld = pd.concat([nc, mci, ad, lbd, vd, prion, ftd, nph, sef, psy, tbi, oc], ignore_index=True)
    df_vld = df_vld.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_vld.drop_duplicates(inplace=True)
    df_vld = df_vld[~df_vld['ID'].isin(list(df_train['ID']))]
    # df_vld.to_csv(save_path, index=False)
    return df_vld

def get_test_split(df, save_path, df_train, df_vld):
    df_test = df[~df['ID'].isin(list(df_train['ID'])) & ~df['ID'].isin(list(df_vld['ID']))]
    df_test = pd.concat([df_test, cli, nacc_np], ignore_index=True).sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_test.drop_duplicates(inplace=True)
    # df_test.to_csv(save_path, index=False)
    return df_test
    

In [ ]:
train = get_train_split(nacc_wo_np_cli, 'training_cohorts/train_vld_test_split/nacc_train.csv')
vld = get_vld_split(nacc_wo_np_cli, 'training_cohorts/train_vld_test_split/nacc_vld.csv', train)
test = get_test_split(nacc_wo_np_cli, 'training_cohorts/train_vld_test_split/nacc_test_with_np_cli.csv', train, vld)

In [ ]:
test['his_TBI'].value_counts()

In [ ]:
len(set(nacc['ID']))

In [ ]:
len(set(train['ID'])) + len(set(vld['ID'])) + len(set(test['ID']))

In [ ]:
len(test['ID'])

In [ ]:
len(vld)

In [ ]:
nacc_lab_dict = {}
nacc_trn_lab_dict = {}
nacc_vld_lab_dict = {}
nacc_tst_lab_dict = {}
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'PDD', 'DLB', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    nacc_lab_dict[label] = nacc[label].value_counts()
    nacc_trn_lab_dict[label] = train[label].value_counts()
    nacc_vld_lab_dict[label] = vld[label].value_counts()
    nacc_tst_lab_dict[label] = test[label].value_counts()
train_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_trn_lab_dict.items()}
vld_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_vld_lab_dict.items()}
test_ratios = {k:(v[1] / nacc_lab_dict[k][1]) for k, v in nacc_tst_lab_dict.items()}

In [ ]:
train_ratios

In [ ]:
vld_ratios

In [ ]:
test_ratios

In [ ]:
len(set(train['ID']).intersection(set(vld['ID'])))

In [ ]:
len(set(vld['ID']) - (set(nacc['ID'])))

In [ ]:
train['his_NACCNIHR'].value_counts()

#### merged

In [ ]:
merged[merged['ID'].isin(nacc_np['ID'])]

In [ ]:
merged_train_vld = merged[~merged['ID'].isin(list(test['ID']))]
n = merged_train_vld['ID'].isna().sum()
replacement_values = range(1, n + 1)
merged_train_vld_rep = copy.deepcopy(merged_train_vld)
merged_train_vld_rep.loc[merged_train_vld_rep['ID'].isna(), 'ID'] = replacement_values

In [ ]:
len(set(merged_train_vld_rep['ID']))

In [ ]:
len(merged_train_vld_rep)

In [ ]:
def get_train_split_merged(df, save_path):
    nc = df[df['NC'] == 1][:int(0.73 * len(df[df['NC'] == 1]))]
    # imci = df[df['IMCI'] == 1][:int(0.75 * len(df[df['IMCI'] == 1]))]
    mci = df[df['MCI'] == 1][:int(0.75 * len(df[df['MCI'] == 1]))]
    # de = nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1][:int(0.8 * len(nacc_unique_df_converted[nacc_unique_df_converted['DE'] == 1]))]
    ad = df[df['AD'] == 1][:int(0.75 * len(df[df['AD'] == 1]))]
    lbd = df[df['LBD'] == 1][:int(0.75 * len(df[df['LBD'] == 1]))]
    vd = df[df['VD'] == 1][:int(0.7 * len(df[df['VD'] == 1]))]
    prion = df[df['PRD'] == 1][:int(0.75 * len(df[df['PRD'] == 1]))]
    ftd = df[df['FTD'] == 1][:int(0.75 * len(df[df['FTD'] == 1]))]
    nph = df[df['NPH'] == 1][:int(0.7 * len(df[df['NPH'] == 1]))]
    sef = df[df['SEF'] == 1][:int(0.7 * len(df[df['SEF'] == 1]))]
    psy = df[df['PSY'] == 1][:int(0.7 * len(df[df['PSY'] == 1]))]
    tbi = df[df['TBI'] == 1][:int(0.6 * len(df[df['TBI'] == 1]))]
    oc = df[df['ODE'] == 1][:int(0.75 * len(df[df['ODE'] == 1]))]

    df_train = pd.concat([nc, mci, ad, lbd, vd, prion, ftd, nph, sef, psy, tbi, oc], ignore_index=True)
    df_train = df_train.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df_train.drop_duplicates(inplace=True)
    df_train_1 = copy.deepcopy(df_train)
    df_train_1.loc[~df_train_1['ID'].astype(str).str.startswith('NACC'), 'ID'] = np.nan
    # df_train_1.to_csv(save_path, index=False)
    # print(df_train_1['ID'])
    return df_train_1, df_train

def get_vld_split_merged(df, save_path, df_train):
    df_vld = df[~df['ID'].isin(list(df_train['ID']))]
    df_vld_1 = copy.deepcopy(df_vld)
    df_vld_1.loc[~df_vld_1['ID'].astype(str).str.startswith('NACC'), 'ID'] = np.nan
    # df_vld_1.to_csv(save_path, index=False)
    return df_vld_1, df_vld

In [ ]:
merged_train_1, merged_train = get_train_split_merged(merged_train_vld_rep, 'training_cohorts/train_vld_test_split/merged_train.csv')
merged_vld_1, merged_vld = get_vld_split_merged(merged_train_vld_rep, 'training_cohorts/train_vld_test_split/merged_vld.csv', merged_train)

In [ ]:
x = merged_train_1[~merged_train_1['ID'].isna()]
x['ID'] = x['ID'].astype(str)
x[x['ID'].str.startswith('1_S')]

In [ ]:
merged_lab_dict = {}
merged_trn_lab_dict = {}
merged_vld_lab_dict = {}
merged_tst_lab_dict = {}
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'PDD', 'DLB', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    merged_lab_dict[label] = merged[label].value_counts()
    merged_trn_lab_dict[label] = merged_train_1[label].value_counts()
    merged_vld_lab_dict[label] = merged_vld_1[label].value_counts()
    merged_tst_lab_dict[label] = test[label].value_counts()
merged_train_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_trn_lab_dict.items()}
merged_vld_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_vld_lab_dict.items()}
merged_test_ratios = {k:(v[1] / merged_lab_dict[k][1]) for k, v in merged_tst_lab_dict.items()}

In [ ]:
len(merged_vld)

In [ ]:
len(set(merged['ID']))

In [ ]:
len(merged_train) + len(merged_vld) + len(test)

In [ ]:
len(merged) + len(set(nacc_np['ID']).union(set(cli['ID'])))

In [ ]:
set(merged['mri_zip']).intersection(set(nacc_np['mri_zip']).union(set(cli['mri_zip'])))

In [ ]:
merged_train_ratios

In [ ]:
merged_vld_ratios

In [ ]:
merged_test_ratios

In [ ]:
len(set(merged_train['ID']).intersection(set(nacc_np['ID'])))

In [ ]:
len(set(merged_vld['ID']).intersection(set(merged_train['ID'])))